In [1]:
import numpy as np
import glob
import os
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
from PIL import Image
import io


Using TensorFlow backend.


In [2]:
#IF YOU DOWNLOADED THE DATA .zip , uncomment and run:

# !pip install pyunpack
# from pyunpack import Archive
# Archive('./data.zip').extractall('.')

# Data Loading /  Pre Processing

In [3]:
DATAPATH = "./data"
train_data_path = "./data/uc_train_256_data"
test_data_path ="./data/uc_test_256"

train_image_paths = sorted(glob.glob(train_data_path + "/*.*"))
test_image_paths = sorted(glob.glob(test_data_path + "/*.*"))

#Partition with image paths
partition = {}
partition['train'] = train_image_paths
partition['validation'] = test_image_paths
               
#complete list of 21 labels' names
lista_labels = [
'agricultural',
'airplane',
'baseballdiamond',
'beach',
'buildings',
'chaparral',
'denseresidential',
'forest',
'freeway',
'golfcourse',
'harbor',
'intersection',
'mediumresidential',
'mobilehomepark',
'overpass',
'parkinglot',
'river',
'runway',
'sparseresidential',
'storagetanks',
'tenniscourt'
]

#Dict for labels, ex: label_dict[path] = 'agricutural'
label_dict = {}
for path in train_image_paths:
    for i in range(len(lista_labels)):
        if lista_labels[i] in path:
            label_dict[path] = lista_labels[i]
for path in test_image_paths:
    for i in range(len(lista_labels)):
        if lista_labels[i] in path:
             label_dict[path] = lista_labels[i]


#Dict for integer class encoding and vice versa, ex: 'agricultural' = 0, 'airplane' = 1

dict_indx_to_labels = {v:k for v, k in enumerate(lista_labels)} 
dict_labels_to_indx = {k:v for v, k in enumerate(lista_labels)}


### Data Generator

In [8]:
import numpy as np
import keras

def Scaling(image):
    return np.array(image) / 255.0

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_paths, labels, batch_size=32, dim=(256,256), n_channels=3,
                 n_classes=21, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_paths = list_paths
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of paths
        list_paths_temp = [self.list_paths[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_paths_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_paths_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, path in enumerate(list_paths_temp):
            with open(path, 'rb') as img_path:                              
                image_PIL = Image.open(io.BytesIO(img_path.read()))
                np_image_scaled = Scaling(image_PIL)
                X[i,] = np_image_scaled#.transpose(2,0,1) #channels first
            # Store class
            y[i] = dict_labels_to_indx[self.labels[path]]
        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [9]:
#Just for debug:
# dataset = DataGenerator(partition['train'], label_dict)
# X, y = dataset.__getitem__(0)

# Model 

In [10]:
def CNN(droprate=0.25, img_rows=256, img_cols=256):
    # input image dimensions
    droprate=droprate
    input_shape = (img_rows, img_cols, 3) #RGB

    #Start Neural Network
    model = Sequential()
    #convolution 1st layer
    model.add(Conv2D(128, kernel_size=(7, 7), padding="same",
                     activation='relu',
                     input_shape=input_shape)) 
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(droprate))
    
    #convolution 2nd layer
    model.add(Conv2D(256, kernel_size=(5, 5), activation='relu',padding="same"))#1
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(droprate))

    #convolution 3rd layer
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu',padding="same"))#1
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(droprate))

    #convolution 4th layer
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu',padding="same"))#1
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(droprate))

    #convolution 5th layer
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',padding="same"))#1
    model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Dropout(droprate))

    #Fully connected 1st layer
    model.add(Flatten()) 
    model.add(Dense(256,use_bias=False)) 
    model.add(BatchNormalization())
    model.add(Activation('relu')) 
    model.add(Dropout(droprate))      

    #Fully connected final layer
    model.add(Dense(21)) #8
    model.add(Activation('softmax')) #9

      
    return model

# Training

In [ ]:
# Generator parameters
params = {'dim': (256,256),
          'batch_size': 8,
          'n_classes': 21,
          'n_channels': 3,
          'shuffle': True}

# Datasets
DATAPATH = "./data"
train_data_path = "./data/uc_train_256_data"
test_data_path ="./data/uc_test_256"

train_image_paths = sorted(glob.glob(train_data_path + "/*.*"))
test_image_paths = sorted(glob.glob(test_data_path + "/*.*"))

partition = {}
partition['train'] = train_image_paths
partition['validation'] = test_image_paths
partition = partition
labels = label_dict

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Training parameters
epochs = 50
droprate=0.25
lr = 0.001

# Design model
model = CNN(droprate)

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=lr),
              metrics=['accuracy'])

#keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

model.summary()

model_path = './best_model.pth'

callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    callbacks = callbacks,
                    epochs=epochs)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 256, 256, 128)     18944     
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 256, 128)     512       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 128, 128)     0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 256)     819456    
_________________________________________________________________
batch_normalization_8 (Batch (None, 128, 128, 256)     1024      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 64, 64, 256)       0         
__________

In [11]:
# #Since I used the validation dataset for test I don't need this for now

# test_images = []
# for idx, filename in enumerate(test_image_paths):
#     with open(filename, 'rb') as img:                              
#             img = Image.open(io.BytesIO(img.read()))
#             test_images.append(img) 
# test_set = np.array([Scaling(test_images[i]) for i in range(len(test_images))])
# X_test_CNN = test_set

# y_test_CNN = to_categorical([dict_labels_to_indx[label_dict[path]] for path in test_image_paths])

# score = model.evaluate(X_test_CNN, y_test_CNN, verbose=0)

# #print loss and accuracy
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])